 # Import the RPB model

In [1]:
from RPB_model import *
from pyomo.environ import *
from pyomo.environ import SolverManagerFactory

# Create a full RPB model (Only need to run once)

In [2]:
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")

# Create Objective Function

In [ ]:
# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.energy_requirement

RPB.objective = Objective(expr=RPB.obj)

RPB.objective.pprint()

### Create solver object (Only need to run once)

In [ ]:
# results = SolverFactory("gams").solve(
#     RPB,
#     tee=True,
#     keepfiles=True,
#     solver="conopt4",
#     tmpdir="temp",
#     add_options=["gams_model.optfile=1;"],
# )

# solver = SolverFactory("ipopt")
# solver.options = {
#         "max_iter": 1000,
#         "bound_push": 1e-8,
#         # "halt_on_ampl_error": "yes",
#         "tol": 1e-4,
#         "max_cpu_time": 5*60,
#         # "mu_strategy": "adaptive",
#         }

# Set Bounds for Decision Variables
##### Should only need to run when initializing the model

In [3]:
RPB.ads.L.setlb(0.01)
RPB.ads.L.setub(40)
RPB.des.L.setlb(0.01)
RPB.des.L.setub(40)

RPB.ads.Tx.setlb(25+273)
RPB.ads.Tx.setub(95+273)

RPB.des.Tx.setlb(100+273)
RPB.des.Tx.setub(160+273)

RPB.ads.P_in.setub(1.5)
RPB.ads.P.setub(1.5)

RPB.des.P_in.setub(1.5)
RPB.des.P_in.setlb(1.01325)
RPB.des.P.setub(1.5)

RPB.ads.w_rpm.setlb(0.00001)
RPB.ads.w_rpm.setub(0.1)

# Initialize Using Json File (90% Capture)
## Only run first time or if solver fails to converge to an optimal solution
#### May have to scroll to see all results

In [4]:
from_json(RPB, fname="polishing step optimized solution 031824.json.gz", gz=True)

{'etime_load_file': 0.13258123397827148,
 'etime_read_dict': 0.3321654796600342,
 'etime_read_suffixes': 0.02155303955078125}

# Run to get RPB Results
### Can copy and paste the table to excel (formatting might just be weird)

In [ ]:
report(RPB)

# Fix decision Variables

In [5]:
# Change Spec of intlet flow to F_in and P_in
RPB.des.F_in.fix()
RPB.des.P_in.fix()
RPB.des.P_out.unfix()
RPB.ads.F_in.fix()
RPB.ads.P_in.fix()
RPB.ads.P_out.unfix()

RPB.ads.L.fix()
RPB.ads.D.fix()
RPB.ads.w_rpm.fix()
RPB.ads.theta.fix()
RPB.des.Tx.fix()

# Ipopt Solver

In [6]:
solver = SolverFactory("ipopt")
solver.options = {
        "max_iter": 1000,
        "bound_push": 1e-8,
        # "halt_on_ampl_error": "yes",
        "tol": 1e-4,
        # "max_cpu_time": 5*60,
        # "mu_strategy": "adaptive",
        }
results = solver.solve(RPB, tee=True)
results.write()

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Ipopt 3.13.2: max_iter=1000
bound_push=1e-08
tol=0.0001


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use 

# Fix Capture Efficiency and Solve

In [ ]:
RPB.ads.CO2_capture.fix(0.9)
# solver = SolverFactory('conopt')
# solver.solve(RPB, tee=True)

# solver.solve(RPB, tee=True)
# results = SolverFactory("gams").solve(
#     RPB,
#     tee=True,
#     keepfiles=True,
#     solver="conopt4",
#     tmpdir="temp",
#     add_options=["gams_model.optfile=1;"],
# )

opt = SolverFactory("gams")
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(
    RPB,
    tee=True,
    # keedfiles=True,
    solver="conopt",
    # tmpdir="temp",
    options={'outlev': 2,
            'workfactor': 3,},
    # add_options=["gams_model.optfile=1;"],
)
results.write()

In [ ]:
import os
email = os.environ.get('NEOS_EMAIL', '')
print(email)
print(type(email))

In [ ]:
RPB.rich_loading_constraint.pprint()

# Check Infeasible Constraints

In [8]:
from pyomo.util.infeasible import find_infeasible_constraints as find_inf_con
i = 0
for c in find_inf_con(RPB):
    i += 1
    print(c[0], c[1])
print(f'There are {i} infeasible constraints')

rich_loading_constraint[0.01] -4.30812355785708e-06
rich_loading_constraint[0.016306894089533095] -4.964128583588479e-06
rich_loading_constraint[0.026591479484724942] -1.216216899730549e-06
rich_loading_constraint[0.04336244396414017] 2.4533834851979243e-06
rich_loading_constraint[0.07071067811865475] 5.459833979625084e-06
rich_loading_constraint[0.11530715390799684] 6.715644059913384e-06
rich_loading_constraint[0.18803015465431966] 4.986690359327218e-06
rich_loading_constraint[0.3066187817586519] 1.5522702428830516e-05
rich_loading_constraint[0.403309] 0.00015071342707070933
rich_loading_constraint[0.5] 0.00017945484087591632
rich_loading_constraint[0.596691] 0.0001772598326278852
rich_loading_constraint[0.6933812182413481] 0.0001864792958951812
rich_loading_constraint[0.8119698453456803] 0.00019339956785318057
rich_loading_constraint[0.8846928460920032] 0.00020517638728820569
rich_loading_constraint[0.9292893218813453] 0.00020502515104845145
rich_loading_constraint[0.9566375560358599

# Change Pressure Drop Scale
#### (1 = Normal dP, 0 = No dP)

In [7]:
import numpy as np
scale = 0.3

for scale in np.linspace(1, 0.1, 15):
    RPB.ads.R_dP.fix(scale)
    RPB.des.R_dP.fix(scale)
    results = solver.solve(RPB, tee=True)
    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        to_json(RPB, fname="polishing_step_reducing_dP.json.gz", gz=True)
        

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Ipopt 3.13.2: max_iter=1000
bound_push=1e-08
tol=0.0001


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use 